<a href="https://colab.research.google.com/github/yohanesnuwara/CO2Inject/blob/master/rock_physics_modelling_co2injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rock Physics Modelling of CO2 Injection in Carbonate

This notebook runs a rock physics modelling to predict the elastic (seismic) response of a carbonate rock to injected CO2. 

# 1. Setting Up

In [0]:
# basic libraries
import numpy as np
import matplotlib as plt

Clone from my github repository: CO2Inject

In [15]:
! git clone https://github.com/yohanesnuwara/CO2Inject/

Cloning into 'CO2Inject'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 53 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [0]:
import Kuster_Toksoz_2

# 2. Compute Properties of Rock Matrix

Properties of rock matrix include: bulk modulus (K), shear modulus (G), and density (rho). 

Input:

In [0]:
rhocalc = 2.71; Kcalc = 76.8; Gcalc = 32
rhoclay = 2.58; Kclay = 20.9; Gclay = 6.9
rhodolo = 2.87; Kdolo = 94.9; Gdolo = 45
rhoqtz = 2.65; Kqtz = 36.6; Gqtz = 45
# mineral composition
calc = 0.4; clay = 0.24; dolo = 0.34; qtz = 0.02

Mineral composition (fraction of calcite, clay, dolomite, and quartz) is estimated from petrophysical analysis (rhomaa, dtmaa, umaa crossplot). [See my another notebook](https://). 

Voigt-Reuss-Hill (VRH) computation:

In [13]:
mincomposition = np.array([calc, clay, dolo, qtz])
rhomineral = np.array([rhocalc, rhoclay, rhodolo, rhoqtz])
Kmineral = np.array([Kcalc, Kclay, Kdolo, Kqtz])
Kmineral_inv = 1 / Kmineral
Gmineral = np.array([Gcalc, Gclay, Gdolo, Kqtz])
Gmineral_inv = 1 / Gmineral

Kv = sum(mincomposition * Kmineral)
Kr_inv = sum(mincomposition * Kmineral_inv)
Gv = sum(mincomposition * Gmineral)
Gr_inv = sum(mincomposition * Gmineral_inv)
Km = (Kv+(1/Kr_inv)) / 2
Gm = (Gv+(1/Gr_inv)) / 2
Km, Gm # in GPa

(58.38150053888909, 24.271778274276585)

# 3. Saturation and Pressure Change

Pressure is an important information for further rock physics modelling. The case that is presented here is **CO2 injection that lasts for 10 years with rate of 800 ton/day of CO2**. Simulation shows that reservoir pressure declines (blue curve). 

![alt text](https://drive.google.com/uc?id=1U0oIG4OCQwvQaCUhr1hFb3AuuQ3jEZ5F)

From simulation, until 10 years, the **maximum CO2 saturation achieves 50%** (high residual water saturation in carbonate system). Pressure declines **from 27.3 MPa to 15.9 MPa**.  

In [0]:
year = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
SW = [1,	0.95,	0.9,	0.85,	0.8,	0.75,	0.7,	0.65,	0.6,	0.55,	0.5] # water saturation
Pp = [27.3,	26.16,	25.02,	23.88,	22.74,	21.6,	20.46,	19.32,	18.18,	17.04,	15.9] # reservoir pressure

# 4. Rock Physics Computation

This will compute how much P-velocity (Vp), S-velocity (Vs), and density of carbonate rock change as effect of CO2 injection from year 1 to year 10. 


Steps:
1. Compute **density and bulk modulus of brine** using Batzle and Wang (1992). Using `Batzle_and_Wang`. Inputs required: reservoir pressure (P), temperature (T), and brine salinity. Result: brine density and bulk modulus computed for each pressure.

2. Compute **density and bulk modulus of CO2** using Span and Wagner (1998). Using `EOS_CO2`. Inputs required: reservoir P, T. Result: CO2 density and bulk modulus computed for each pressure.

3. Mixing brine and CO2, using **Brie (intermediate patchy) mixing** because of vug porosity. For each composition `Sw` brine and `(1-Sw)` CO2. Result: density and bulk modulus of brine + CO2

4. **Kuster-Toksoz (1974) modelling** to compute Vp, Vs, and density. 

The implementation is as follows: 